In [1]:
%pip install -r requirements.txt 

  Using cached lap-0.4.0.tar.gz (1.5 MB)
  Preparing metadata (setup.py) ... done
  Using cached numpy-2.1.3-cp310-cp310-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached opencv_python_headless-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl.metadata (19 kB)
INFO: pip is looking at multiple versions of sahi to determine which version is compatible with other requirements. This could take a while.
  Using cached imutils-0.5.4-py3-none-any.whl
  Using cached filterpy-1.4.5-py3-none-any.whl
ERROR: Cannot install -r requirements.txt (line 1), numpy==2.1.3 and sahi==0.11.18 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested numpy==2.1.3
    filterpy 1.4.5 depends on numpy
    sahi 0.11.18 depends on numpy<2.0.0

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip to attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https:/

In [4]:
%pip install sahi==0.11.18

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 720.8 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 398.8 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.12.0.88
    Uninstalling opencv-python-4.12.0.88:
      Successfully uninstalled opencv-python-4.12.0.88
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [sahi]3/7 [opencv-python]
Note: you may need to restart the kernel to use updated packages.


In [13]:
%pip install scikit-image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 476.5 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-image] [scikit-image]
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install  imutils==0.5.4

  Using cached imutils-0.5.4.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'imutils' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'imutils'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25868 sha256=12a6c92db1d6dd08bb4dc4edf3263a0fda7c27c75221f75878ddb85c83460374
  Stored in directory: /Users/user/Library/Caches/pip/wheels/85/cf/3a/e265e975a1e7c7e54eb3692d6aa4e2e7d6a3945d29da46f2d7
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


In [4]:
from sahi import AutoDetectionModel
from sahi.predict import get_prediction
from sahi.utils.cv import read_image
import cv2
import imutils
from sort import *
import time

# Load the YOLOv8 model
yolov8_model_path = "yolov8n.pt"
detection_model = AutoDetectionModel.from_pretrained(
    model_type="yolov8",
    model_path=yolov8_model_path,
    confidence_threshold=0.35,
    device= 'cpu' #"cuda:0" 
)

# Load your video file
video_path = 'Computer Vision task Culture Hint.mp4'
video = cv2.VideoCapture('test2.mp4')

# Initialize the SORT tracker for object tracking
tracker = Sort(max_age=20, min_hits=2, iou_threshold=0.3)

# Define the upper and lower limits for counting objects
limitsEntry = [10, 410, 790, 410]
limitsExit = [10, 370, 790, 370]

# Initialize lists to keep track of counted objects
total_countEntry = []
total_countExit = []

# Iterate over video frames
while True:
    ret, frame = video.read()
    if not ret:
        break
    
    # Resize the frame
    frame = imutils.resize(frame, width=800, height=480)

    # Increase Contrast and Brightness
    frame = cv2.convertScaleAbs(frame, alpha=1.5, beta=0.5)

    # Perform prediction on the frame
    result = get_prediction(frame, detection_model)

    # Initialize detections array
    detections = np.empty((0, 5))

    for obj in result.object_prediction_list:
        if obj.category.name != "person":
            continue
        bbox = obj.bbox.to_voc_bbox()
        detections = np.vstack((detections, [bbox[0], bbox[1], bbox[2], bbox[3], obj.score.value]))

    # Update the tracker with the new detections
    resultsTracker = tracker.update(detections)

    # Draw bounding boxes and labels on the frame
    for obj in resultsTracker:
        x1, y1, x2, y2, id = obj # Get coordinates and ID
        w, h = x2 - x1, y2 - y1  # Calculate width and height


        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2) # Draw bounding boxes
        label = f"Person #{id}"
        cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)


        # Calculate the center of the bounding box
        cx, cy =int(x1 + w // 2), int(y1 + h // 2)
        cv2.circle(frame, (cx, cy), 5, (255, 0, 0), -1)  # Draw a circle at the center
        
        # Check if the center of the object crosses the upper limit
        if limitsEntry[0] < cx < limitsEntry[2] and limitsEntry[1] - 15 < cy < limitsEntry[1] + 15:
            if total_countEntry.count(id) == 0:  # Only count if not counted before
                total_countEntry.append(id)  # Add ID to the count list
        
        # Check if the center of the object crosses the lower limit
        if limitsExit[0] < cx < limitsExit[2] and limitsExit[1] - 15 < cy < limitsExit[1] + 15:
            if total_countExit.count(id) == 0:  # Only count if not counted before
                total_countExit.append(id)  # Add ID to the count list
        

        # Draw lines for counting limits
        cv2.line(frame, (limitsEntry[0], limitsEntry[1]), (limitsEntry[2], limitsEntry[3]), (139, 195, 75), thickness=2)
        cv2.line(frame, (limitsExit[0], limitsExit[1]), (limitsExit[2], limitsExit[3]), (50, 50, 230), thickness=2)

        # Draw Heading for Entry and Exit
        cv2.putText(frame, "Entry", (10, 410), cv2.FONT_HERSHEY_PLAIN, 0.7, (139, 195, 75), 2)
        cv2.putText(frame, "Exit", (10, 370), cv2.FONT_HERSHEY_PLAIN, 0.7, (50, 50, 230), 2)


    # Display the total count of objects that crossed the upper and lower limits
    cv2.putText(frame, "ENTRY : " + str(len(total_countEntry)), (20, 40), cv2.FONT_HERSHEY_PLAIN, 3, (139, 195, 75), 2)
    cv2.putText(frame, "EXIT : " +str(len(total_countExit)), (20, 100), cv2.FONT_HERSHEY_PLAIN, 3, (50, 50, 230), 2)


    # Display the frame
    cv2.imshow("People Entry/Exit Counting System", frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the video and close all windows
video.release()
cv2.destroyAllWindows()

In [15]:
%pip install filterpy

  Using cached filterpy-1.4.5.zip (177 kB)
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'filterpy' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'filterpy'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110547 sha256=c51caa94d09c9f3438bbf30370c17e9aad9d5e5d00f5fe43e25890490a0bb47e
  Stored in directory: /Users/user/Library/Caches/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy
Note: you may need to restart the kernel to use updated packages.
